<a href="https://colab.research.google.com/github/marcosfeandrade/EEG_BCI_Analysis/blob/main/EEG_BCI_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.utils import to_categorical

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.utils import class_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [22]:
def compile_model():
  model = models.Sequential()
  model.add(Dense(22, input_dim=22))
  model.add(Activation('sigmoid'))
  model.add(Dense(4))
  model.add(Activation('sigmoid'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])
  return model

In [23]:
def evaluate_model(testX,testY,history,model):
  print("plotando gráficos...")
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [24]:
def sliding_window(dataInput, window_size, window_skip):
  data = dataInput.drop(columns=['patient','time','epoch'], axis=1)
  df_new = pd.DataFrame(columns=data.columns)
  i = 0
  rms_array = np.zeros(shape=data.columns.size - 1)
  while(i + window_size <= len(data)):
    df_range = data.iloc[i:i + window_size]
    df_columns_range = df_range.drop(columns='label', axis=1)
    classes = df_range['label']
    for j in range(0, 22):
      values_array = df_columns_range.iloc[:, j].values
      rms_array[j] = np.sqrt(np.mean(np.square(values_array)))
    rms_array = np.append(rms_array, classes[classes.last_valid_index()])
    result = pd.DataFrame(rms_array.reshape(1, -1), columns=data.columns)
    rms_array = np.zeros(shape=data.columns.size - 1)
    df_new = df_new.append(result, ignore_index=True)
    i = i + window_skip
  return df_new

In [25]:
import pandas as pd
def move_column_to_end(df, movedColumn):
  if movedColumn in df.columns:
    # Crie uma lista com a ordem desejada das colunas
    nova_ordem_colunas = [col for col in df.columns if col != movedColumn] + [movedColumn]
    # Reorganize as colunas do DataFrame de acordo com a nova ordem
    df = df[nova_ordem_colunas]
  return df

In [26]:
path = '/content/drive/My Drive/BCICIV_patients/BCICIV_2a_all_patients.csv'
window_size = 200
window_skip = 100
df_read = pd.read_csv(path)
df_moved = move_column_to_end(df_read, 'label')

In [ ]:
df_new = sliding_window(df_moved, window_size, window_skip)

In [ ]:
target_count = df_new["label"].value_counts()
target_count.plot(kind='bar', title='Dataset com todas as label de sinal');

In [ ]:
# Excluindo registros com a classe 'tongue'
df_new = df_new[df_new["label"] != "tongue"]

# Unificando as classes 'left' e 'right' em uma classe 'arms'
df_new.loc[(df_new["label"] == "left") | (df_new["label"] == "right"), "label"] = "arms"

# Atualizando os rótulos restantes
df_new.loc[df_new["label"] == "foot", "label"] = "1"
df_new["label"] = df_new["label"].map({"arms": "0", "foot": "1"})

In [30]:
min_max_scaler = preprocessing.MinMaxScaler()
X = df_new.drop(columns='label', axis=1)
x_scaled = min_max_scaler.fit_transform(X.values)
X = pd.DataFrame(x_scaled)
lb = LabelEncoder()
labels = lb.fit_transform(df_new['label'])
y = np.asarray(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train),
                                                 y = y_train)
class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}
y_train = to_categorical(y_train, target_count.count())
y_test = to_categorical(y_test, target_count.count())

In [ ]:
new_series = pd.Series(y_train.sum(axis=0))
new_series.plot(kind='bar', title='Quantidade de elementos por classes para o treino')

In [ ]:
new_series = pd.Series(y_test.sum(axis=0))
new_series.plot(kind='bar', title='Quantidades de elementos por classes para o teste');

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Criando e treinando um modelo Random Forest
modelo_rf = RandomForestClassifier(n_estimators=5000, random_state=60, class_weight='balanced_subsample', verbose=1, max_depth=100)
modelo_rf.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred_rf = modelo_rf.predict(X_test)

# Avaliando o desempenho do modelo
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f'Acurácia do Random Forest: {acc_rf:.4f}\n')

# Imprimindo o relatório de classificação
print('Relatório de Classificação:\n', classification_report(y_test, y_pred_rf))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Calcular a matriz de confusão multi-label
ml_cm = multilabel_confusion_matrix(y_test, y_pred_rf)

# Consolidar as matrizes de confusão em uma única matriz
consolidated_cm = np.sum(ml_cm, axis=0)

# Plotar a matriz de confusão consolidada
plt.figure(figsize=(10,7))
sns.heatmap(consolidated_cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão Consolidada')
plt.ylabel('Rótulo Verdadeiro')
plt.xlabel('Rótulo Previsto')
plt.show()


# **OUTROS TESTES**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

accuracies = []
n_trees = [10, 50, 100, 500, 1000, 2000, 5000]  # Número de árvores para testar

for n in n_trees:
    model = RandomForestClassifier(n_estimators=n, random_state=60, class_weight='balanced_subsample', max_depth=100)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

# Plotando o gráfico
plt.figure(figsize=(10, 6))
plt.plot(n_trees, accuracies, marker='o')
plt.xlabel('Número de Árvores')
plt.ylabel('Acurácia')
plt.title('Acurácia do RandomForest em função do Número de Árvores')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Substitua com o caminho correto do seu arquivo
path = '/content/drive/My Drive/BCICIV_patients/BCICIV_2a_all_patients.csv'

# Carregando o dataset
df = pd.read_csv(path)

# Filtrando as linhas e unindo 'left' e 'right' em 'arms'
df_filtered = df[df['label'].isin(['right', 'left', 'foot'])]
df_filtered['label'] = df_filtered['label'].replace(['left', 'right'], 'arms')

# Separando características e rótulos
X = df_filtered.drop('label', axis=1)
y = df_filtered['label']

# Normalizando os dados
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Codificando os rótulos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividindo em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Treinando o modelo RandomForest
model = RandomForestClassifier(n_estimators=2000, random_state=60, class_weight='balanced_subsample', verbose=1, max_depth=100)
history = model.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay

# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calculando a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Exibindo a matriz de confusão como um heatmap
labels = label_encoder.classes_
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Exibindo a matriz de confusão como um gráfico
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()
plt.show()


In [ ]:
# Fazendo previsões
predictions = model.predict(X_test)

# Avaliando o desempenho do modelo
acc_rf = accuracy_score(y_test, predictions)
print(f'Acurácia do Random Forest: {acc_rf:.4f}\n')

# Imprimindo o relatório de classificação
print('Relatório de Classificação:\n', classification_report(y_test, predictions))

In [39]:
# Imprimindo as previsões e os valores reais para as primeiras 20 amostras
for i in range(300):
    previsao = label_encoder.inverse_transform([predictions[i]])[0]
    valor_real = label_encoder.inverse_transform([y_test[i]])[0]
    print(f'Previsão: {previsao}, Valor Real: {valor_real}')

Previsão: arms, Valor Real: foot
Previsão: arms, Valor Real: foot
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: foot, Valor Real: foot
Previsão: arms, Valor Real: foot
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: foot
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: foot
Previsão: arms, Valor Real: foot
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: foot
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: foot
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: arms, Valor Real: arms
Previsão: 